In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import os
import requests
import seaborn as sns

class myclass():
    def __init__(self, model_stock):
        self.model_stock = model_stock
        
        
    def main(self):
        #全時系列データの呼び出し
        path = '../../stock_data/time_series_data/{}'.format(self.model_stock)
        stock_datas = os.listdir(path)
        count = 0
        data = pd.DataFrame()
        cols = ['return','sma30','sma90','sma150','sma210','sma270'

                        ,'ratio_sma75_30', 'ratio_sma75_150', 'ratio_sma105_30', 
                        'ratio_sma105_150', 'ratio_sma135_30', 'ratio_sma135_150'

                       ,'Highest81','Highest81,30days_ago','Highest121','Highest121,30days_ago'
                        ,'Highest121,60days_ago','Standard_deviation_normalization30',
                       'Standard_deviation_normalization40'
                       ,'adosc','price-change', 'price-change-percentage', 'volume', 'amount', 'sma10', 'sma10-FP',
        'macd', 'macd-SG', 'macd-histogram', 'cci-SG', 'mtm10', 'roc-SG', 'roc-FP', 'rsi', 'rsi-FP',
            'adosc', 'adosc-SG', 'ar26', 'br26', 'bias20',]
#                 print(df)
        for i in stock_datas:
            
            count = count +1
            print(count)
            print(i)
#             try:
            df = pd.read_csv('{}/{}/'.format(path,self.model_stock) + i)
            df = df.set_index('Date')
            #範囲を多めに取って計算量を減らす
            df = df['2017-01-01':'2021-3-31']

            df = create_data(df)


            print(df.isnull().any())
            data = data.append(df)



                
#             except:
#                 continue
#             # save_path = '/content/drive/MyDrive/LSTM/model1/model/preprocess_data/{}/'.format(self.model_stock)
#             save_path = '/content/drive/MyDrive/LSTM/model1/backtest/preprocess_data/{}/'.format(self.model_stock)
#             save_csv(df[cols], i, save_path)

        
        for i in cols:
            
            x = data[i]
            sns.distplot(x)


def create_data(df):
    # feature calculation
    # basic information
    df['price-change'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df['price-change-percentage'] = df['Adj Close'] / df['Adj Close'].shift(1)

    df['amount'] = df['Adj Close'] * df['Volume']

    # simple moving average
    df['sma10'] = df['Adj Close'].rolling(30).mean()
    df['sma10-FP'] = (df['sma10'] - df['sma10'].shift(1)) / df['sma10'].shift(1)
 
    # Moving Average Convergence Divergence
    df['macd'] = df['Adj Close'].rolling(12).mean() - df['Adj Close'].rolling(26).mean()
    df['macd-SG'] = df['macd'].rolling(9).mean()
    df['macd-histogram'] = df['macd'] - df['macd-SG']
    df['macd-histogram'] = np.where(df['macd-histogram'] > 0, 1, -1)
    df['macd-SG'] = np.where(df['macd-SG'] > 0, 1, -1)
    df['macd'] = np.where(df['macd'] > 0, 1, -1)
    # Commodity Channel Index in 24 days
    df['typical-price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['sma-cci'] = df['typical-price'].rolling(24).mean()
    df['mean-deviation'] = np.abs(df['typical-price'] - df['sma-cci'])
    df['mean-deviation'] = df['mean-deviation'].rolling(24).mean()
    df['cci'] = (df['typical-price'] - df['sma-cci']) / (0.015 * df['mean-deviation'])
    df['cci-SG'] = np.where(df['cci'] > 0, 1, -1)
    # MTM 10
    df['mtm10'] = df['Adj Close'] - df['Adj Close'].shift(10)
    df['mtm10'] = np.where(df['mtm10'] > 0, 1, -1)
    # Rate of Change in 10 days
    df['roc'] = (df['Adj Close'] - df['Adj Close'].shift(10)) / df['Adj Close'].shift(10)
    df['roc-SG'] = np.where(df['roc'] > 0, 1, -1)
    df['roc-FP'] = (df['roc'] - df['roc'].shift(1))

    # Relative Strength Index in 5 days
    df['rsi'] = df['price-change'].rolling(5).apply(RSI) / 100
    df['rsi-FP'] = (df['rsi'] - df['rsi'].shift(1))

    # Slow K and Slow D
#     df['slow-k'] = df['Adj Close'].rolling(14).apply(SlowK)
#     df['slow-d'] = df['slow-k'].rolling(14).mean()
#     df['slow-k-FP'] = df['slow-k'] - df['slow-k'].shift(1)
#     df['slow-d-FP'] = df['slow-d'] - df['slow-d'].shift(1)
#     df['slow-k'] = Zero_One_Scale(df['slow-k'])
#     df['slow-d'] = Zero_One_Scale(df['slow-d'])
#     df['slow-k-FP'] = One_One_Scale(df['slow-k-FP'])
#     df['slow-d-FP'] = One_One_Scale(df['slow-d-FP'])
    # ADOSC
    df['adosc'] = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
    df['adosc'] = df['adosc'].cumsum()
    df['adosc-ema3'] = df['adosc'].ewm(span=3, adjust=False).mean()
    df['adosc-ema10'] = df['adosc'].ewm(span=10, adjust=False).mean()
    df['adosc-SG'] = np.where((df['adosc-ema3'] - df['adosc-ema10']) > 0, 1, -1)

    # AR 26
    hp_op = (df['High'] - df['Open']).rolling(26).sum()
    op_lp = (df['Open'] - df['Low']).rolling(26).sum()
    df['ar26'] = hp_op / op_lp

    # BR 26
    hp_cp = (df['High'] - df['Close']).rolling(26).sum()
    cp_lp = (df['Close'] - df['Low']).rolling(26).sum()
    df['br26'] = hp_cp / cp_lp

    # VR 26
    
    # BIAS 20
    sma20 = df['Adj Close'].rolling(20).mean()
    df['bias20'] = (df['Adj Close'] - sma20) / sma20
    df['bias20'] = np.where(df['bias20'] > 0, 1, -1)
    
    
    
    df['price-change'] = One_One_Scale(df['price-change'])
    # set return and direction (label)
    df['return'] = np.log(df['Adj Close'].shift(-5) / df['Adj Close'])
    df['direction'] = np.where(df['return'] > 0, 1, -1)
    df['direction'] = df['direction'].shift(-1)
    
    AD = []
    AD.append(0)
    for i in range(1, len(df)):
        AD_component = ((df["Adj Close"][i] - df["Low"][i]) - (df["High"][i] - df["Adj Close"][i])) * df['Volume'][i] / (df["High"][i] - df["Low"][i]) + AD[-1]
        AD.append(AD_component)
    df['A/D'] = AD
    df['A/D_EMA'] = df['A/D'].ewm(com=20).mean()   
    df['A/D_ratio'] = df['A/D'] / df['A/D_EMA']
    df['ADOSC'] = df['A/D'].ewm(com=3).mean() / df['A/D'].ewm(com=10).mean()   
    df['A/D_EMA_ratio'] = df['A/D_EMA'] / df['A/D_EMA'].shift(1)
                
#     df['A/D'] = Zero_One_Scale(df['A/D'])
#     df['A/D_EMA'] = Zero_One_Scale(df['A/D_EMA'])
#     df['A/D_ratio'] = Zero_One_Scale(df['A/D_ratio'])
#     df['A/D_EMA_ratio'] = Zero_One_Scale(df['A/D_ratio'])
#     df['ADOSC'] = Zero_One_Scale(df['ADOSC'])
   
    
    
    OBV = []
    OBV.append(0)
    for i in range(1, len(df.Close)):
        if df.Close[i] > df.Close[i-1]: #If the closing price is above the prior close price 
              OBV.append(OBV[-1] + df.Volume[i]) #then: Current OBV = Previous OBV + Current Volume
        elif df.Close[i] < df.Close[i-1]:
              OBV.append( OBV[-1] - df.Volume[i])
        else:
              OBV.append(OBV[-1])
                
    #Store the OBV and OBV EMA into new columns
    df['OBV'] = OBV
    df['OBV_EMA'] = df['OBV'].ewm(com=20).mean()   
    df['OBV_ratio'] = df['OBV'] / df['OBV_EMA']
    
    df['OBV_EMA_ratio'] =  df['OBV_EMA'] / df['OBV_EMA'].shift(1)
                
#     df['OBV'] = Zero_One_Scale(df['OBV'])
#     df['OBV_EMA'] = Zero_One_Scale(df['OBV_EMA'])
#     df['OBV_ratio'] = Zero_One_Scale(df['OBV_ratio'])
#     df['OBV_EMA_ratio'] = Zero_One_Scale(df['OBV_EMA_ratio'])

     # Commodity Channel Index in 24 days
    df['typical-price'] = (df['High'] + df['Low'] + df['Close']) / 3
   
    

    # simple moving average
    for i in [30,45,60,90,120,150,180,200,210,240,270,300]:
        df['sma'+str(i)] = df['Adj Close'].rolling(i).mean()
        df['sma'+str(i)] = df['sma'+str(i)] / df['sma'+str(i)].shift(1)
#         df['sma'+str(i)] = Zero_One_Scale(df['sma'+str(i)])
    for i in [30,60,90,120,150,180,200,210,240,270,300]:
        df["ema"+str(i)]=df["Adj Close"].ewm(span=i).mean()
        df["ema"+str(i)] = df["ema"+str(i)] / df["ema"+str(i)].shift(1)
#         df["ema"+str(i)] = Zero_One_Scale(df["ema"+str(i)])
    
    
    #calucurate aroon
    for periods in [14,20]:
        df['aroon_up'+str(periods)] = df['High'].rolling(periods+1).apply(lambda x: x.argmax(), raw=True) / periods * 100
        df['aroon_down'+str(periods)] = df['Low'].rolling(periods+1).apply(lambda x: x.argmin(), raw=True) / periods * 100
        df['aroon_ratio'+str(periods)] = df['aroon_up'+str(periods)] / df['aroon_down'+str(periods)]
#         df['AROONOSC'+str(periods)] = df['aroon_up'+str(periods)] - df['aroon_down'+str(periods)]

        df['aroon_up'+str(periods)] = Zero_One_Scale(df['aroon_up'+str(periods)])
        df['aroon_down'+str(periods)] = Zero_One_Scale(df['aroon_down'+str(periods)])
        df['aroon_ratio'+str(periods)] = Zero_One_Scale(df['aroon_up'+str(periods)])
#         df['AROONOSC'+str(periods)] = Zero_One_Scale(df['AROONOSC'+str(periods)])


     #calucurate ADX
    df["TrueRange"] = np.nan
    df["PDM"] = np.nan
    df["NDM"] = np.nan
    for i in range(1,len(df)):
        df["TrueRange"][i] = TrueRange(df["Adj Close"][i],df["High"][i],df["Low"][i],df["Open"][i],df["Adj Close"][i-1])
        df["PDM"][i] = PDM(df["Open"][i],df["High"][i],df["Low"][i],df["Adj Close"][i],df["Open"][i-1],df["High"][i-1],df["Low"][i-1],df["Adj Close"][i-1])
        df["NDM"][i] = NDM(df["Open"][i],df["High"][i],df["Low"][i],df["Adj Close"][i],df["Open"][i-1],df["High"][i-1],df["Low"][i-1],df["Adj Close"][i-1])
    
    df['PDI'] = df["PDM"].rolling(14).sum()/df["TrueRange"].rolling(14).sum() * 100
    df['NDI'] = df["NDM"].rolling(14).sum()/df["TrueRange"].rolling(14).sum() * 100

    
    df['DX'] = (df['PDI']-df['NDI']).abs()/(df['PDI']+df['NDI']) * 100
    df['DX'] = df['DX'].fillna(0)
    
    df['ADX'] = df['DX'].rolling(14).mean()
    df['ADXR'] = df['ADX'].rolling(14).mean()
    
#     df['ADX'] = Zero_One_Scale(df['ADX'])
#     df['ADXR'] = Zero_One_Scale(df['ADXR'])
    
    
    for i in range(15,150,30):
        for k in range(30,270,60):
            df['ratio_sma'+str(k)] = df['Adj Close'].rolling(k).mean()
            df['ratio_sma'+str(i)] = df['Adj Close'].rolling(i).mean()
            df['ratio_sma'+str(i)+'_'+str(k)] = df['ratio_sma'+str(i)] / df['ratio_sma'+str(k)]
#             df['ratio_sma'+str(i)+'_'+str(k)] = Zero_One_Scale(df['ratio_sma'+str(i)])

            
    for term in range(5,50,5):
        df['SMA'+str(term)] = df['Adj Close'].rolling(term).mean()
        df['STD'+str(term)] = df['Adj Close'].rolling(term).std()
        df['Standard_deviation_normalization'+str(term)] = 100 * 2 * df['STD'+str(term)] / df['SMA'+str(term)]
#         df['Standard_deviation_normalization'+str(term)] = Zero_One_Scale(df['Standard_deviation_normalization'+str(term)])
        
    for i in [15,45,81,121,161]:
        df['Highest'+str(i)] = df['Adj Close'].rolling(window=81).max()
        df['Highest'+str(i)] = df['Highest'+str(i)].shift()
        for m in [30,90,150,60,60]:
            df['Highest'+str(i)+','+str(m)+'days_ago'] = df['Adj Close'] / df['Highest'+str(i)].shift(m)
#             df['Highest'+str(i)+','+str(m)+'days_ago'] = Zero_One_Scale(df['Highest'+str(i)+','+str(m)+'days_ago'])
        
        #今日の終値が過去何日間の高音に対してどの程度あるか
        df['Highest'+str(i)] = df['Adj Close'] / df['Highest'+str(i)]
#         df['Highest'+str(i)] = Zero_One_Scale(df['Highest'+str(i)])
        
    # ADOSC
    df['adosc'] = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
    df['adosc'] = df['adosc'].cumsum()
#     df['adosc'] = Zero_One_Scale(df['adosc'])
        


    validation(df)
    # drop row contains NaN
    df.dropna(inplace=True)
    # adjust the length of the data, which should be a multiple number of 30.
    length = len(df) // 5
    
    return df[:5 * length]


    

    return df




def validation(df):
    k=5
    df["Open"] = df["Open"].shift(-1)
    df["validation"] = np.nan
    for i in range(0,len(df)-k):

        df["validation"][i] = df['Adj Close'][i+k]/df['Open'][i]

#     df["validation"] = Zero_One_Scale(df["validation"])

    return df

def TrueRange(c, h, l, o, yc):
    x = h-l
    y = abs(h-yc)
    z = abs(l-yc)
    if y <= x >= z:
        TR = x
    elif x <= y >= z:
        TR = y
    elif x <= z >= y:
        TR = z
    return TR

def PDM(o, h, l, c, yo, yh, yl, yc):
    moveUp = h - yh
    moveDown = yl - l
    if 0 < moveUp > moveDown:
        PDM = moveUp
    else:
        PDM = 0
        
    return PDM

def NDM(o, h, l, c, yo, yh, yl, yc):
    moveDown = yl - l
    moveUp = h - yh
    if 0 < moveDown > moveUp:
        NDM = moveDown
    else:
        NDM = 0
    
    return NDM


    
    
    

def save_csv(df, sym, save_path):
    sym = sym.replace('.T', '')
    df.to_csv(save_path+'{}'.format(sym))
    print('{}.csv Saved'.format(sym))

    
    
myclass = myclass('nasdaq100')
myclass.main()





1
CSCO.csv


FileNotFoundError: [Errno 2] No such file or directory: '../../stock_data/time_series_data/nasdaq100/nasdaq100/CSCO.csv'